# Sprint 機械学習スクラッチ 決定木

## 【考察】

* どう決定木は作られていくか。
* 以下の条件次第で、木の構成は変わる。
    * 学習方法
    * ハイパーパラメータ
    * 訓練データ

* 今回の決定木は量的変数のみに特化する。
    * カテゴリ変数には「0と1」で代用する。
    


In [4]:
import numpy as np

In [1]:
class ScratchDecesionTreeClassifierDepth1():
    """
    深さ1の決定木分類器のスクラッチ実装

    Parameters
    ----------
    verbose : bool
      学習過程を出力する場合はTrue
    """
    def __init__(self, verbose=False):
        # ハイパーパラメータを属性として記録
        self.verbose = verbose
    def fit(self, X, y):
        """
        決定木分類器を学習する
        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
            訓練データの特徴量
        y : 次の形のndarray, shape (n_samples, )
            訓練データの正解値
        """
        if self.verbose:
            #verboseをTrueにした際は学習過程を出力
            print()
        pass
    def predict(self, X):
        """
        決定木分類器を使いラベルを推定する
        """
        pass
        return

## 【問題1】不純度を求める関数（CART式）

### ノードのジニ不純度を計算する関数を作成してください

* ジニ不純度とは、そのノードでのサンプルのクラスの異なりが同程度存在する確率。
    * 確率が高いとノード内のサンプルが全て、異なるクラスに属している。
        * データが半々なのは悪い分類
    * 確率が低いとノード内のサンプルが全て、同じクラスに属している。
    * ベルヌーイ分布における、全てのクラスの分散の和に相当する。
* ノード内の不純度を最大限減らす（ジニ不純度が低い）素性と閾値の組を選ぶために、ジニ不純度を用いる。
* 不純度が最も低ければジニ不純度の値は0、不純度が高くなればなるほどジニ不純度の値が1に漸近する。（[参照先url](https://qiita.com/3000manJPY/items/ef7495960f472ec14377)）
* 最終的に情報利得Δgainで算出する。
    * 利得が高い特徴と閾値ほど、不純度を最大限減らせる。
    


1. ジニ係数を算出する関数を構築する。
2. ジニ係数を用い、情報利得を算出する関数を構築する。


In [86]:
def gini_coef(*args):
    """""
    ジニ係数を算出する。
    
    Parameters
    ----------
    *args : int
        根ノード内の各特徴量毎のサンプル数（値）を渡す。
        
    return
    ----------
    ジニ係数
    """""
    sample_all = sum(args)
    gini_coef_answer = 0
    for i in range(len(args)):
        gini_coef_answer += np.power(args[i]/sample_all, 2)
    return 1 - gini_coef_answer

In [88]:
# クラス1:サンプル数15, クラス2:サンプル数15 → ジニ不純度0.500
print(f'例題1のジニ不純度: {gini_coef(15, 15)}')
print()
# クラス1:サンプル数15, クラス2:サンプル数15, クラス3:サンプル数15 → ジニ不純度0.667
print(f'例題2のジニ不純度: {gini_coef(15, 15, 15) :.2f}')
print()
# クラス1:サンプル数18, クラス2:サンプル数12 → ジニ不純度0.480
print(f'例題3のジニ不純度: {gini_coef(18, 12)}')
print()
# クラス1:サンプル数30, クラス2:サンプル数0 → ジニ不純度0.000
print(f'例題4のジニ不純度: {gini_coef(30, 0)}')

例題1のジニ不純度: 0.5

例題2のジニ不純度: 0.67

例題3のジニ不純度: 0.48

例題4のジニ不純度: 0.0


## 【問題2】情報利得を求める関数

* 問題1で算出した確率はジニ不純度（ジニ係数）$I(t)$をroot_node $I(p)$として用いる。
* 左右各ノードのサンプル数を引数として情報利得を算出する。

In [47]:
def info_gain(left_node_1, left_node_2, right_node_1, right_node_2):
    """""
    情報利得を算出する。
    
    Parameters
    ----------
    left_node_1 : int
        左ノード内の第1特徴量のサンプル数（値）を渡す。
    left_node_2 : int
        左ノード内の第2特徴量のサンプル数（値）を渡す。
    right_node_1 : int
        右ノード内の第1特徴量のサンプル数（値）を渡す。
    right_node_2 : int
        右ノード内の第2特徴量のサンプル数（値）を渡す。
        
    return
    ----------
    gain : float
        情報利得
    """""    
    sample_all_list = [left_node_1, left_node_2, right_node_1, right_node_2] # パラメータのリスト化
    sample_all = sum(sample_all_list) # 全サンプルの総和
    
    left_all = np.add(left_node_1, left_node_2) # 左分岐の総和
    # print(f'left_all: {left_all}')
    right_all = np.add(right_node_1, right_node_2) # 右分岐の総和
    # print(f'right_all: {right_all}')
    root_node = [left_node_1 + right_node_1, left_node_2 + right_node_2] # 根ノードの左右総和数をリスト化
    # print(f'root_node: {root_node}')
    
    gini_coef_answer = 0 
    for i in range(len(root_node)):
        gini_coef_answer += np.power(root_node[i]/sample_all, 2)
    gini_coef = 1 - gini_coef_answer # ジニ係数
    # print(f'gini_coef: {gini_coef}')
    
    left_node_coef = 1 - (np.power(left_node_1/left_all, 2) + np.power(left_node_2/left_all, 2)) # 左ノードのジニ係数
    # print(f'left_node_coef: {left_node_coef}')
    right_node_coef = 1 - (np.power(right_node_1/right_all, 2) + np.power(right_node_2/right_all, 2)) # 右ノードのジニ係数
    # print(f'right_node_coef: {right_node_coef}')
    gain = gini_coef - ((left_all/sample_all) * left_node_coef) - ((right_all/sample_all) * right_node_coef) # 情報利得の算出
    
    # print(f'左辺: {(left_all/sample_all) * left_node_coef}')
    # print(f'右辺: {(right_all/sample_all) * right_node_coef}')
    return gain

In [51]:
# test
# 左ノードクラス1:サンプル数10, 左ノードクラス2:サンプル数30, 右ノードクラス1:サンプル数20, 右ノードクラス2:サンプル数5 → 情報利得0.143
answer = info_gain(10, 30, 20, 5)
print('例題の情報利得：0.143')
print(f'スクラッチ関数の利得：{answer:.3f}')

例題の情報利得：0.143
スクラッチ関数の利得：0.143


## 【問題3】学習

1. 全サンプルの情報利得を算出する。
    0. 説明変数Xと目的変数yを結合させる。
    1. 根ノードの説明変数Xを、目的変数yを基に分別する。
        * 目的変数0 : 左
        * 説明変数1 : 右
    2. 説明変数Xから、行列内のインデックスを一つ抽出する。
    3. 抽出した引数を閾値として、葉ノードに振り分ける。
        * 閾値以下：左
        * 閾値以上：右
    4. 左右の葉ノード内も目的変数を基に分別する。
        * 目的変数0 : 左
        * 説明変数1 : 右
    5. 問題2の関数を用いて、情報利得を算出する。
        * その関数に必要なパラメータである、葉ノード内の分別済説明変数（サンプル数）を引用する。
    6. 算出した情報利得を新しい行列内に追記する。
    7. ステップBに戻り、次のインデックスを抽出する。（繰り返す）
    8. 全てのインデックスを基に算出した情報利得を代入した行列を完成させる。
2. 最大値の情報利得を抽出する。
    1. numpyの特性上、情報利得を代入した行列内の各列毎の最大値を抽出する。
    2. 各列毎の最大値の中で、最も高い数値の要素とインデックスを抽出する。
    3. 抽出した要素とインデックスをインスタンス化させて、次の推定に活用する。
    
    
    
    閾値をとりあえず設定する。
    データを2領域に分割する
    分割した領域の中でラベル事の個数を求める。
    利得を求める。
    以降、繰り返す。

In [60]:
# Sample data
X = np.array([[1, 2],[3, 4], [5, 6], [7, 8], [9, 10]])
y = np.array([0, 1, 1, 0, 1]).reshape(5, 1)
y.shape

(5, 1)

In [ ]:
def fit(self, X, y):
    """
    決定木分類器を学習する
    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
        訓練データの特徴量
    y : 次の形のndarray, shape (n_samples, )
        訓練データの正解値
    """

    root_node_array = np.append(X, y, axis=1) # 横(axis=1)に結合させる
    
    # 1_A
    root_node_0 = root_node_array[:, np.all(root_node_array[:, 3] == 0, axis=0)]
        
    
    root_node_0 = left_node_1 + right_node_1
    root_node_1 = left_node_2 + right_node_2
    
    root_node = [root_node_0, root_node_1] # 根ノードの左右総和数をリスト化
    # print(f'root_node: {root_node}')
    
    
    if self.verbose:
        #verboseをTrueにした際は学習過程を出力
        print()
    
    
    
    
    pass

In [35]:
root_node = [40, 25]
sample_all = sum(root_node)
gini_coef_answer = 0 

for i in range(len(root_node)):
    gini_coef_answer += np.power(root_node[i]/sample_all, 2)
gini_coef = 1 - gini_coef_answer # ジニ係数

gini_coef

0.47337278106508873

In [75]:
# Sample data
X = np.array([[1, 2],[3, 4], [5, 6], [7, 8], [9, 10]])
y = np.array([0, 1, 1, 0, 1]).reshape(5, 1)
y.shape

(5, 1)

In [119]:
root_node_array = np.append(X, y, axis=1)
root_node_array

array([[ 1,  2,  0],
       [ 3,  4,  1],
       [ 5,  6,  1],
       [ 7,  8,  0],
       [ 9, 10,  1]])

In [129]:
root_node_0 = np.any(root_node_array[:, 2:3] == 0, axis=1)
root_node_0

array([ True, False, False,  True, False])

In [108]:
root_node_array

array([[ 1,  2,  0],
       [ 3,  4,  1],
       [ 5,  6,  1],
       [ 7,  8,  0],
       [ 9, 10,  1]])